In [14]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import json
from pathlib import Path
from sklearn.linear_model import LinearRegression

# %%
# Load the data into a Pandas DataFrame
stock_df = pd.read_csv(
    "../Resources/data.csv",
    index_col="Company")

# %%
# Display sample data
stock_df.head(10)

# %%
# Remove the $ sign from the columns 
stock_df = stock_df.replace({'\$': ' '}, regex=True)

# Review the changes 
stock_df.head(5)

# %%
stock_df.info()

# %%
# Check for missing values
null_counts = stock_df.isnull().sum()
print(null_counts)

# %%
# Convert to datetime
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Find the oldest and newest dates
oldest_date = stock_df['Date'].min()
newest_date = stock_df['Date'].max()

print(oldest_date)
print(newest_date)

# %%
# Sort the df by company and date 
stock_df.sort_values(by=['Company', 'Date'], inplace=True)
stock_df.head(5)

# %%
# Calculating the change between closing and opening prices per company
stock_df['Close/Last'] = pd.to_numeric(stock_df['Close/Last'], errors='coerce')
stock_df['Open'] = pd.to_numeric(stock_df['Open'], errors='coerce')

stock_df['Change'] = stock_df.groupby('Company')['Close/Last'].shift(1) - stock_df['Open']
stock_df.head(5)

# %%
# Delete null value in Change column
stock_df.dropna(subset=['Change'], inplace=True)
stock_df.head(5)


<class 'pandas.core.frame.DataFrame'>
Index: 25160 entries, AAPL to NFLX
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        25160 non-null  object
 1   Close/Last  25160 non-null  object
 2   Volume      25160 non-null  int64 
 3   Open        25160 non-null  object
 4   High        25160 non-null  object
 5   Low         25160 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB
Date          0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
dtype: int64
2013-07-18 00:00:00
2023-07-17 00:00:00


,Date,Close/Last,Volume,Open,High,Low,Change
Company,,,,,,,
AAPL,2013-07-19,15.1768,268548901,15.4679,15.4993,15.1554,-0.0480
AAPL,2013-07-22,15.2254,207648981,15.3379,15.3482,15.1953,-0.1611
AAPL,2013-07-23,14.9639,354477618,15.2143,15.2486,14.9539,0.0111
AAPL,2013-07-24,15.7325,591624923,15.6761,15.8782,15.545,-0.7122
AAPL,2013-07-25,15.6607,229432412,15.7393,15.7643,15.5646,-0.0068


In [3]:

# %%
# Filter for AMZN data only
amzn_data = stock_df[stock_df.index == 'AMZN']

amzn_data.head()


,Date,Close/Last,Volume,Open,High,Low,Change
Company,,,,,,,
AMZN,2013-07-19,15.2615,59250980,15.2195,15.2895,15.0955,-0.014
AMZN,2013-07-22,15.1740,50411260,15.4625,15.4625,15.10,-0.201
AMZN,2013-07-23,15.0530,39981880,15.1580,15.253,15.028,0.016
AMZN,2013-07-24,14.9470,39980680,15.1510,15.192,14.902,-0.098
AMZN,2013-07-25,15.1700,107373640,14.9500,15.225,14.8375,-0.003


In [13]:

# %%
# Plot AMZN data
amzn_plot = amzn_data.hvplot.scatter(
    x="Open",
    y="Close/Last",
    title="AMZN Open vs Close prices"
)
amzn_plot


:Scatter   [Open]   (Close/Last)

In [5]:

# %%
# Reformat data of the independent variable X as a single-column array
X = amzn_data["Open"].values.reshape(-1, 1)

# Display sample data
X[:5]


array([[15.2195],
       [15.4625],
       [15.158 ],
       [15.151 ],
       [14.95  ]])

In [6]:

# %%
# The shape of X is 30 samples, with a single feature (column)
X.shape

# %%
# Create an array for the dependent variable y
y = amzn_data["Close/Last"]

# %%
# Create a model with scikit-learn
model = LinearRegression()

# %%
# Fit the data into the model
model.fit(X, y)


LinearRegression()

In [7]:

# %%
# Display the slope
print(f"Model's slope: {model.coef_}")

# %%
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

# %%
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

# %%
# Display the formula to predict the close price if the open price is $30
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 30")


Model's slope: [0.99862364]
Model's y-intercept: 0.07811837970122326
Model's formula: y = 0.07811837970122326 + 0.9986236443997035X
Model's formula: y = 0.07811837970122326 + 0.9986236443997035 * 30


In [16]:

# predict the close price if the open price is $30
y_30 = model.intercept_ + model.coef_[0] * 30

# Display the prediction
print(f"Predicted stock closing price if the open price is $30: ${y_30:.2f}")


Predicted stock closing price if the open price is $30: $30.04


In [9]:

# %%
# Make predictions using the X set
predicted_y_values = model.predict(X)


In [17]:

# %%
# Create a copy of the original data
df_close_predicted = amzn_data.copy()

# Add a column with the predicted salary values
df_close_predicted["close_predicted"] = predicted_y_values

# Display sample data
df_close_predicted.head()

# %%
# Create a line plot of the predicted salary values
best_fit_line = df_close_predicted.hvplot.line(
    x = "Open",
    y = "close_predicted",
    color = "red"
)
best_fit_line


:Curve   [Open]   (close_predicted)

In [18]:

# %%
# Superpose the original data and the best fit line
amzn_plot * best_fit_line


:Overlay
   .Scatter.I :Scatter   [Open]   (Close/Last)
   .Curve.I   :Curve   [Open]   (close_predicted)

In [12]:

# %%
# Import relevant metrics - score, r2, mse, rmse, std - from Scikit-learn
from sklearn.metrics import mean_squared_error, r2_score

# %%
# Compute the metrics for the linear regression model
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9990380003633789.
The r2 is 0.9990380003633789.
The mean squared error is 2.585193436934862.
The root mean squared error is 1.607853673981206.
The standard deviation is 51.83929282012655.


In [15]:
models_info = {}
# Save model information
model_info = {
    "intercept": model.intercept_,
    "coef": model.coef_[0]
}
# Store the model information in the dictionary
models_info['AMZN'] = model_info
# Save the model information to a JSON file named after the ticker symbol
filename = f"{'AMZN'}_model_info.json"
with open(filename, "w") as json_file:
    json.dump(model_info, json_file)